In [1]:
# !kaggle datasets download -d puneet6060/intel-image-classification

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import albumentations as A
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.utils import image_dataset_from_directory, plot_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom, Dense, Flatten, Dropout, Conv2D, MaxPooling2D

In [2]:
# from zipfile import ZipFile
# file_name="./dataset/intel-image-classification.zip"

# with ZipFile(file_name,'r') as zip:
#     zip.extractall()

In [2]:
# Set dataset path
train_path = './dataset/seg_train/seg_train/'
test_path = './dataset/seg_test/seg_test/'

## CNN Model

In [3]:
def get_CNN_model():
    # CNN model architecture
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(6, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [4]:
# Load dataset to type tensorflow PrefetchDataset
def load_train_validation_dataset(path, seed):
    train_ds = image_dataset_from_directory(
        directory=path,
        seed=seed,
        validation_split=0.2,
        subset="training",
        image_size=(150, 150),
        shuffle=True,
    )
    val_ds = image_dataset_from_directory(
        directory=path,
        seed=seed,
        validation_split=0.2,
        subset="validation",
        image_size=(150, 150),
        shuffle=True,
    )
    return train_ds, val_ds

In [5]:
# Load dataset to type tensorflow PrefetchDataset
def load_test_dataset(path, seed):
    test_ds = image_dataset_from_directory(
        directory=path,
        image_size=(150, 150),
        shuffle=False
    )
    return test_ds

In [6]:
def preprocessing(train_ds, val_ds, test_ds):
    # Rescale pixel value from 0~255 to 0~1
    scaling = tf.keras.layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (scaling(x), y))
    val_ds = val_ds.map(lambda x, y: (scaling(x), y))
    test_ds = test_ds.map(lambda x, y: (scaling(x), y))

    # Make training process faster
    train_ds = train_ds.cache().shuffle(buffer_size=64).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().shuffle(buffer_size=64).prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return train_ds, val_ds, test_ds

In [7]:
# get test dataset's label from PrefetchDataset and return numpy array
def get_test_label(test_ds):
    test_data = [(image.numpy(), label.numpy()) for image, label in test_ds]
    test_label = test_data[0][1]
    for i in range(1, len(test_data)):
        test_label = np.concatenate([test_label, test_data[i][1]], axis=0)
    
    return list(test_label)

In [8]:
# turn probability prediction to label
def get_prediction_label(prediction):
    predict_label = np.argmax(prediction, axis=1)
    
    return list(predict_label)

In [9]:
def get_metric_results(test_label, predict_label):
    metric_results = {}
    metric_results['accuracy'] = accuracy_score(test_label, predict_label)
    metric_results['f1_score'] = f1_score(test_label, predict_label, average="macro")
    metric_results['precision'] = precision_score(test_label, predict_label, average="macro")
    metric_results['recall'] = recall_score(test_label, predict_label, average="macro")
    return metric_results

In [11]:
accuracy = []
f1score = []
precision = []
recall = []

for i in range(5):
    seed = i ** 2
    train_ds, val_ds = load_train_validation_dataset(train_path, seed)
    test_ds = load_test_dataset(test_path, seed)
    train_ds, val_ds, test_ds = preprocessing(train_ds, val_ds, test_ds)
    
    model = get_CNN_model()
    model.fit(train_ds,
              validation_data=val_ds,
              epochs=10)
    
    prediction = model.predict(test_ds)
    
    test_label = get_test_label(test_ds)
    predict_label = get_prediction_label(prediction)
    
    metric_result = get_metric_results(test_label, predict_label)
    
    accuracy.append(metric_result['accuracy'])
    f1score.append(metric_result['f1_score'])
    precision.append(metric_result['precision'])
    recall.append(metric_result['recall'])

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Found 14034 files belonging to 6 classes.
Using 2806 files for validation.
Found 3000 files belonging to 6 classes.
Epoch 1/10
351/351 [==============================] - 5s 13ms/step - loss: 1.0712 - accuracy: 0.5765 - val_loss: 0.7987 - val_accuracy: 0.7014
Epoch 2/10
351/351 [==============================] - 3s 10ms/step - loss: 0.7705 - accuracy: 0.7162 - val_loss: 0.7235 - val_accuracy: 0.7452
Epoch 3/10
351/351 [==============================] - 3s 9ms/step - loss: 0.6471 - accuracy: 0.7646 - val_loss: 0.5853 - val_accuracy: 0.7947
Epoch 4/10
351/351 [==============================] - 3s 9ms/step - loss: 0.5515 - accuracy: 0.8015 - val_loss: 0.5524 - val_accuracy: 0.8097
Epoch 5/10
351/351 [==============================] - 3s 9ms/step - loss: 0.4752 - accuracy: 0.8293 - val_loss: 0.6374 - val_accuracy: 0.7812
Epoch 6/10
351/351 [==============================] - 3s 9ms/step - loss: 0.4019 - accuracy: 0.856

In [23]:
print('Accuracy:', ['%.2f' % val for val in accuracy])
print('F1score:', ['%.2f' % val for val in f1score])
print('Precision:', ['%.2f' % val for val in precision])
print('Recall:', ['%.2f' % val for val in recall])
print('Average accuracy:', sum(accuracy) / len(accuracy))
print('Average f1score:', sum(f1score) / len(f1score))
print('Average precision:', sum(precision) / len(precision))
print('Average recall:', sum(recall) / len(recall))

Accuracy: ['0.81', '0.81', '0.83', '0.83', '0.82']
F1score: ['0.82', '0.81', '0.83', '0.83', '0.82']
Precision: ['0.82', '0.81', '0.84', '0.83', '0.82']
Recall: ['0.82', '0.81', '0.83', '0.83', '0.82']
Average accuracy: 0.82
Average f1score: 0.821073622157208
Average precision: 0.825036380671853
Average recall: 0.8212334485802897


## Transfer Learning VGG

In [16]:
def get_VGG16_pretrained_model():
    pretrained_model = VGG16(input_shape = (150, 150, 3), 
                             include_top = False, 
                             weights = 'imagenet')

    for layer in pretrained_model.layers:
         layer.trainable = False
    
    return pretrained_model

In [17]:
def get_transfered_model(last_layer_output, pretrained_model_input):
    x = Flatten()(last_layer_output)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)                  
    x = Dense(6, activation='softmax')(x)           
    model = Model(pretrained_model_input, x)
    
    model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
    
    return model

In [18]:
VGG_accuracy = []
VGG_f1score = []
VGG_precision = []
VGG_recall = []

pretrained_model = get_VGG16_pretrained_model()
last_layer = pretrained_model.get_layer('block5_pool')

for i in range(5):
    seed = i ** 2
    train_ds, val_ds = load_train_validation_dataset(train_path, seed)
    test_ds = load_test_dataset(test_path, seed)
    train_ds, val_ds, test_ds = preprocessing(train_ds, val_ds, test_ds)
    
    model = get_transfered_model(last_layer.output, pretrained_model.input)
    
    model.fit(train_ds,
              validation_data=val_ds,
              epochs=5)
    
    prediction = model.predict(test_ds)
    
    test_label = get_test_label(test_ds)
    predict_label = get_prediction_label(prediction)
    
    metric_result = get_metric_results(test_label, predict_label)
    
    VGG_accuracy.append(metric_result['accuracy'])
    VGG_f1score.append(metric_result['f1_score'])
    VGG_precision.append(metric_result['precision'])
    VGG_recall.append(metric_result['recall'])

Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Found 14034 files belonging to 6 classes.
Using 2806 files for validation.
Found 3000 files belonging to 6 classes.
Epoch 1/5
351/351 [==============================] - 18s 44ms/step - loss: 0.5057 - accuracy: 0.8125 - val_loss: 0.3751 - val_accuracy: 0.8635
Epoch 2/5
351/351 [==============================] - 11s 33ms/step - loss: 0.3383 - accuracy: 0.8782 - val_loss: 0.3627 - val_accuracy: 0.8724
Epoch 3/5
351/351 [==============================] - 11s 33ms/step - loss: 0.2755 - accuracy: 0.8986 - val_loss: 0.3516 - val_accuracy: 0.8778
Epoch 4/5
351/351 [==============================] - 11s 33ms/step - loss: 0.2337 - accuracy: 0.9165 - val_loss: 0.3554 - val_accuracy: 0.8849
Epoch 5/5
94/94 [==============================] - 3s 36ms/step
Found 14034 files belonging to 6 classes.
Using 11228 files for training.
Found 14034 files belonging to 6 classes.
Using 2806 files for validation.
Found 3000 files belongin

In [24]:
print('Accuracy:', ['%.2f' % val for val in VGG_accuracy])
print('F1score:', ['%.2f' % val for val in VGG_f1score])
print('Precision:', ['%.2f' % val for val in VGG_precision])
print('Recall:', ['%.2f' % val for val in VGG_recall])
print('Average accuracy:', sum(VGG_accuracy) / len(VGG_accuracy))
print('Average f1score:', sum(VGG_f1score) / len(VGG_f1score))
print('Average precision:', sum(VGG_precision) / len(VGG_precision))
print('Average recall:', sum(VGG_recall) / len(VGG_recall))

Accuracy: ['0.88', '0.88', '0.88', '0.87', '0.85']
F1score: ['0.88', '0.89', '0.88', '0.88', '0.85']
Precision: ['0.88', '0.89', '0.89', '0.88', '0.88']
Recall: ['0.88', '0.89', '0.88', '0.88', '0.85']
Average accuracy: 0.8735333333333333
Average f1score: 0.8759250985478586
Average precision: 0.8823271380573475
Average recall: 0.8764593897398842
